<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/lava-language-model/lava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
class LM(nn.Module):
  
  def __init__(self, hid_size, vocab_size, n_head, n_layers, max_len, device):
    super().__init__()

    self.device = device
    
    self.hid_size = hid_size
    self.max_len = max_len

    self.embedding = nn.Embedding(vocab_size, hid_size)

    self.position_enc = nn.Embedding(self.max_len, self.hid_size)
    self.position_enc.weight.data = self.position_encoding_init(self.max_len, self.hid_size)
    self.scale = torch.sqrt(torch.FloatTensor([self.hid_size])).to(device)

    self.layer_norm = nn.LayerNorm(self.hid_size)
    self.decoder_layer = nn.TransformerDecoderLayer(d_model=hid_size, nhead = n_head)
    self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=n_layers, norm=self.layer_norm)
    self.fc = nn.Linear(hid_size, vocab_size)

    self._init_weights()
  
  def forward(self, x):
    sent_len, batch_size = x.shape[0], x.shape[1]
    memory_mask = self.generate_complete_mask(sent_len)
    tgt_mask = self.generate_triangular_mask(sent_len)
    memory = torch.zeros(1, batch_size, self.hid_size, device=self.device)

    temp = x
    temp = self.embedding(temp)

    pos = torch.arange(0,sent_len).unsqueeze(1).repeat(1,batch_size).to(self.device)
    temp_pos_emb = self.position_enc(pos)

    temp = temp * self.scale + temp_pos_emb
    temp = self.decoder(temp, memory, tgt_mask=tgt_mask)
    temp = self.fc(temp)
    return temp

  def _init_weights(self):
    for p in self.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

  def generate_triangular_mask(self, size):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        return
        
  def generate_complete_mask(self, size):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = torch.empty(size, size).to(device)
        mask.fill_(float('-inf'))
        return mask

  def generate_sequence(self, src):
    #src = [sent_len]
    src = src.unsqueeze(1)
    #src = [sent_len, 1]
    generate_step = 0
    while generate_step < 10:
      out = self.forward(src)
      #out = [sent_len + 1, 1, vocab_size]
      out = torch.argmax(out[-1, :], dim=1) # [1]
      out = out.unsqueeze(0) #[1,1]
      src = torch.cat((src, out), dim=0)
      generate_step += 1
    src = src.squeeze(1)
    return src
  
  def position_encoding_init(self, n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    temp = torch.from_numpy(position_enc).type(torch.FloatTensor)
    temp = temp.to(self.device)
    return temp

In [0]:
import torchtext
from torchtext import data
import spacy
 
my_tok = spacy.load('en')
 
def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]
 
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [0]:
from torchtext.datasets import LanguageModelingDataset
my_dataset = LanguageModelingDataset("./text.txt", TEXT)

In [195]:
TEXT.build_vocab(my_dataset)
print(len(TEXT.vocab))

2931


In [322]:
train_iter = data.BPTTIterator(
    my_dataset,
    batch_size=1024,
    bptt_len=11, # this is where we specify the sequence length
    device=device,
    repeat=False,
    shuffle=True)

print(len(train_iter))

2


In [0]:
vocab_size = len(TEXT.vocab)
hid_size = 16
model = LM(hid_size, vocab_size, 4, 1, 32, device).to(device)

In [198]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 237,011 trainable parameters


In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [0]:
optimizer = NoamOpt(hid_size, 1, 2000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = torch.nn.CrossEntropyLoss()

In [324]:
from tqdm import tqdm

clip = 1

N_EPOCH = 200
for epoch in range(N_EPOCH):
  epoch_loss = 0
  model.train()
  for batch in train_iter:
    optimizer.zero_grad()
    batch_text = batch.text
    batch_target = batch.target
    result = model(batch_text)
    loss = criterion(result.view(-1, result.shape[-1]), batch_target.view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  print("epoch is {} loss is {}".format(epoch, epoch_loss / len(train_iter)))

epoch is 0 loss is 2.511195659637451
epoch is 1 loss is 2.5533971786499023
epoch is 2 loss is 2.4878326058387756
epoch is 3 loss is 2.54245126247406
epoch is 4 loss is 2.5498387813568115
epoch is 5 loss is 2.516339123249054
epoch is 6 loss is 2.55051052570343
epoch is 7 loss is 2.4868292212486267
epoch is 8 loss is 2.5083754658699036
epoch is 9 loss is 2.4708719849586487
epoch is 10 loss is 2.4456741213798523
epoch is 11 loss is 2.4036545157432556
epoch is 12 loss is 2.393104612827301
epoch is 13 loss is 2.422989308834076
epoch is 14 loss is 2.3539364337921143
epoch is 15 loss is 2.3010640740394592
epoch is 16 loss is 2.325761139392853
epoch is 17 loss is 2.292555570602417
epoch is 18 loss is 2.2681416273117065
epoch is 19 loss is 2.223766803741455
epoch is 20 loss is 2.1749913096427917
epoch is 21 loss is 2.1705713868141174
epoch is 22 loss is 2.1531930565834045
epoch is 23 loss is 2.0723487734794617
epoch is 24 loss is 2.092807173728943
epoch is 25 loss is 2.049001395702362
epoch is 

In [332]:
source_sentence = ["the"]
print(source_sentence)
model.eval()
print(' '.join(source_sentence))
print()
x = TEXT.numericalize([source_sentence]).to(device).squeeze(1)
generated_sequence =model.generate_sequence(x)
words = [TEXT.vocab.itos[word_idx] for word_idx in generated_sequence]
print(' '.join(words))

['the']
the

the one <eos> language is like your design is the <eos>
